<a href="https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials-HIDDEN/blob/main/CompressedCosmos_Saturn_rise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy
!pip install ephem
!pip install timezonefinder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.7 MB/s eta 0:00:00


In [3]:
import datetime
import math

import ephem
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from pytz import timezone

def saturn_rise_time(location, obs_time=None):
    """Calculates the rising time of Saturn for a given location and time.

    Args:
        location (str or tuple): City name (str) or (latitude, longitude) tuple.
        obs_time (datetime, optional): Observation time (defaults to current time).

    Returns:
        datetime: Saturn's rising time (local time).
        None: If Saturn is not above the horizon during the specified time.
    """

    # Determine observer location and timezone
    if isinstance(location, str):
        geolocator = Nominatim(user_agent="saturn_rise_calculator")
        location = geolocator.geocode(location)
        if not location:
            raise ValueError(f"Invalid city name: '{location}'")
        observer = ephem.Observer()
        observer.lat = str(location.latitude)
        observer.lon = str(location.longitude)
        print(f"Corresponding long / lat values: {math.degrees(observer.lon)},"
                                               f"{math.degrees(observer.lat)}")
    elif isinstance(location, tuple):
        observer = ephem.Observer()
        observer.lat, observer.lon = location
    else:
        raise ValueError("Invalid location format. Use city name or (latitude, longitude).")

    tf = TimezoneFinder(in_memory=True)  # For faster lookups
    tz_name = tf.timezone_at(lng=location.longitude, lat=location.latitude)
    obs_timezone = timezone(tz_name) if tz_name else timezone('UTC')

    # Set observation time
    if obs_time is None:
        obs_time = datetime.datetime.now(obs_timezone)
    observer.date = obs_time

    # Calculate Saturn's rising time (more precise)
    saturn = ephem.Saturn(obs_time)
    try:
        rising_time_utc = observer.next_rising(saturn, use_center=True)
        rising_time_local = ephem.localtime(rising_time_utc).astimezone(obs_timezone)
        return rising_time_local
    except ephem.CircumpolarError:
        return None  # Saturn is circumpolar

# Get user input
while True:
    location_input = input("Enter city name or latitude,longitude: ")
    try:
        if "," in location_input:
            lat, lon = map(float, location_input.split(","))
            location = (lat, lon)
        else:
            location = location_input
        break
    except ValueError:
        print("Invalid input format. Use city name or latitude,longitude.")

# Calculate and display result
rise_time = saturn_rise_time(location)
if rise_time:
    print(f"Saturn rises at {rise_time.strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("Saturn is not visible above the horizon during the specified time.")

Enter city name or latitude,longitude: New York City
Corresponding long / lat values: -74.0060152,40.7127281
Saturn rises at 2024-06-30 23:59:16
